In [29]:
import pandas as pd
import numpy as np
from sklearn.mixture import GaussianMixture
from sklearn.metrics import accuracy_score,confusion_matrix, plot_confusion_matrix, plot_roc_curve, recall_score, f1_score, precision_score
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

In [18]:
df = pd.read_parquet('./Data/draft0522.parquet')

In [19]:
model_df = df.drop(['school_name'],axis=1)

In [20]:
model_df.Round[model_df.Round <= 3] = 1

C:\Users\capta\AppData\Local\Temp\ipykernel_9604\88595506.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df.Round[model_df.Round <= 3] = 1


In [21]:
model_df.Round[model_df.Round > 3] = 0

C:\Users\capta\AppData\Local\Temp\ipykernel_9604\1331494837.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df.Round[model_df.Round > 3] = 0


In [12]:
model_df.Round.value_counts(normalize=True)

0.0    0.707763
1.0    0.292237
Name: Round, dtype: float64

In [22]:
X = model_df.drop(['Round'],axis=1)
y = model_df.Round

In [37]:
X_train_num

,height,weight,forty_yd,vertical,bench_reps,broad_jump,cone,shuttle,g_season,def_int_season,...,tackles_assists_career,tackles_total_career,tackles_loss_career,sacks_career,pass_defended_career,fumbles_forced_season,fumbles_forced_career,Age,speed,agility
2503,73.0,298.0,5.18,NaN,41.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.0,82.780343,NaN
4791,78.0,312.0,5.27,32.5,19.0,116.0,7.71,4.71,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0,80.898809,126.794659
214,76.0,262.0,4.84,29.5,22.0,108.0,7.62,4.50,10.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0,95.488229,127.785399
1614,76.0,315.0,5.16,29.5,15.0,100.0,7.93,4.93,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.867250,106.648039
4722,73.0,285.0,5.08,31.0,32.0,115.0,7.63,4.62,12.0,NaN,...,43.0,83.0,22.0,13.5,4.0,NaN,1.0,NaN,85.589405,125.114301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3772,76.0,216.0,4.62,31.5,16.0,124.0,7.06,NaN,14.0,1.0,...,48.0,164.0,11.5,3.0,11.0,1.0,2.0,22.0,94.823470,NaN
5191,71.0,235.0,NaN,NaN,NaN,NaN,NaN,NaN,11.0,NaN,...,128.0,267.0,42.0,14.5,8.0,NaN,NaN,NaN,NaN,NaN
5226,75.0,238.0,4.74,29.0,NaN,113.0,NaN,4.49,7.0,1.0,...,97.0,244.0,22.5,8.0,10.0,NaN,3.0,24.0,94.296069,117.117445
5390,70.0,179.0,4.34,42.5,13.0,136.0,6.84,4.12,11.0,1.0,...,26.0,108.0,4.5,1.0,11.0,NaN,2.0,NaN,100.907654,124.249361


In [58]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.33,random_state=42)

player_name = X_test.player
player_pick = X_test.pick
player_year = X_test.year
player_team = X_test.team

X_train.drop(['player','pick','year','team'],axis=1,inplace=True)
X_test.drop(['player','pick','year','team'],axis=1,inplace=True)

X_train_cat = X_train.select_dtypes('object')
X_train_num = X_train.select_dtypes('float64')

cat_transformer = Pipeline(steps=[('ohe',OneHotEncoder(drop='first',sparse=False,handle_unknown='ignore'))])
num_transformer = Pipeline(steps=[
    ('ss',StandardScaler()),
    ('impute',SimpleImputer(strategy='constant'))])

transformer = ColumnTransformer(transformers=[
    ('num', num_transformer, X_train_num.columns),
    ('cat', cat_transformer, X_train_cat.columns)])

pipe = Pipeline(steps=[
    ('trans',transformer),
    ('gaus',GaussianMixture(10,random_state=42))
])

pipe.fit(X_train,y_train)

Pipeline(steps=[('trans',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('ss',
                                                                   StandardScaler()),
                                                                  ('impute',
                                                                   SimpleImputer(strategy='constant'))]),
                                                  Index(['height', 'weight', 'forty_yd', 'vertical', 'bench_reps', 'broad_jump',
       'cone', 'shuttle', 'g_season', 'def_int_season', 'def_int_yds_season',
       'def_int_yds_per_int_season', 'def_int_career', 'def_int_yds_...
       'tackles_loss_career', 'sacks_career', 'pass_defended_career',
       'fumbles_forced_season', 'fumbles_forced_career', 'Age', 'speed',
       'agility'],
      dtype='object')),
                                                 ('cat',
                                                  Pip

In [59]:
len(pipe.feature_names_in_)

84

In [64]:
y_pred = pipe.predict_proba(X_test)

C:\Users\capta\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [65]:
pd.DataFrame(y_pred)

,0,1,2,3,4,5,6,7,8,9
0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,1.000000e+00
1,0.0,0.0,0.0,0.000000e+00,1.000000e+00,0.000000e+00,0.0,8.441145e-43,0.0,0.000000e+00
2,0.0,0.0,0.0,0.000000e+00,0.000000e+00,1.000000e+00,0.0,8.474093e-28,0.0,0.000000e+00
3,0.0,0.0,0.0,0.000000e+00,0.000000e+00,1.000000e+00,0.0,1.439031e-18,0.0,0.000000e+00
4,0.0,0.0,0.0,2.952718e-107,0.000000e+00,0.000000e+00,0.0,0.000000e+00,1.0,2.738174e-79
...,...,...,...,...,...,...,...,...,...,...
1998,0.0,0.0,0.0,6.864627e-18,0.000000e+00,0.000000e+00,0.0,0.000000e+00,1.0,0.000000e+00
1999,0.0,0.0,0.0,9.516394e-21,0.000000e+00,0.000000e+00,0.0,0.000000e+00,1.0,0.000000e+00
2000,0.0,0.0,0.0,0.000000e+00,3.042176e-304,1.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00
2001,0.0,0.0,0.0,0.000000e+00,0.000000e+00,6.330225e-184,0.0,1.000000e+00,0.0,0.000000e+00
